In [ ]:
!pip install langchain-community pypdf

In [ ]:
!pip install sentence-transformers

In [ ]:
!pip install chromadb

In [ ]:
!pip install langchain_huggingface

In [5]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Load Document

In [6]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader('/content/drive/My Drive/code/Generate/glasses.pdf')

data = loader.load()
print(data)

[Document(metadata={'producer': 'Skia/PDF m136 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'glasses', 'source': '/content/drive/My Drive/code/Generate/glasses.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='Title:  The  Beginner’s  Guide  to  Keeping  Your  Glasses  Clean  (and  Actually  Seeing  Clearly)  \n1.  Introduction  \nGlasses  are  like  windows  for  your  eyes—except  they  somehow  attract  more  smudges  than  a  \ntoddler’s\n \nhands\n \non\n \na\n \ntouchscreen.\n \nWhether\n \nyou\n \nwear\n \nthem\n \nfor\n \nstyle\n \nor\n \nsurvival,\n \nkeeping\n \nthem\n \nclean\n \nand\n \nwell-maintained\n \nis\n \nessential\n \nfor\n \nboth\n \nyour\n \nvision\n \nand\n \nyour\n \nsanity.\n \n2.  How  to  Properly  Clean  Your  Glasses  \nSpoiler  alert:  Your  T-shirt  is  not  the  right  tool.  Here’s  what  to  do  instead:  \nThe  Right  Way:  \n1.  Rinse  First  –  Run  your  glasses  under  lukewarm  water  to  remove  dus

## Parsing Document

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
import re
separator = r"\d+\."

chunk_size = 200
chunk_overlap = 50

rc_splitter = RecursiveCharacterTextSplitter(
        separators=separator,
		chunk_size=chunk_size,
		chunk_overlap=chunk_overlap)

docs = rc_splitter.split_documents(data)

In [8]:
print([doc.page_content for doc in docs[0:3]])

['Title:  The  Beginner’s  Guide  to  Keeping  Your  Glasses  Clean  (and  Actually  Seeing  Clearly)  \n1.  Introduction  \nGlasses  are  like  win', 'duction  \nGlasses  are  like  windows  for  your  eyes—except  they  somehow  attract  more  smudges  than  a  \ntoddler’s\n \nhan', 'dges  than  a  \ntoddler’s\n \nhands\n \non\n \na\n \ntouchscreen.\n \nWhether\n \nyou\n \nwear\n \nthem\n \nfor\n \nstyle\n \nor\n \nsurvival,\n \nkeeping\n \nthem\n \nclean\n \nand\n \nwell-maintaine']


In [9]:
print(f'Character count for each chunk: {[len(doc.page_content) for doc in docs[0:3]]}')

Character count for each chunk: [144, 127, 166]


In [10]:
# Iterate over docs and clean the \n in the text content (page_content)
cleaned_docs = []
for doc in docs:
    # Clean the \n in the page_content of each document
    cleaned_text = doc.page_content.replace('\n', ' ')
    # Create a new Document with the cleaned content
    cleaned_docs.append(doc.copy(update={'page_content': cleaned_text}))

print([cleaned_doc.page_content for cleaned_doc in cleaned_docs[0:3]])

['Title:  The  Beginner’s  Guide  to  Keeping  Your  Glasses  Clean  (and  Actually  Seeing  Clearly)   1.  Introduction   Glasses  are  like  win', 'duction   Glasses  are  like  windows  for  your  eyes—except  they  somehow  attract  more  smudges  than  a   toddler’s   han', 'dges  than  a   toddler’s   hands   on   a   touchscreen.   Whether   you   wear   them   for   style   or   survival,   keeping   them   clean   and   well-maintaine']


<ipython-input-10-9224ce36f7bb>:7: PydanticDeprecatedSince20: The `copy` method is deprecated; use `model_copy` instead. See the docstring of `BaseModel.copy` for details about how to handle `include` and `exclude`. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  cleaned_docs.append(doc.copy(update={'page_content': cleaned_text}))


## Initialize Embedding Model

In [160]:
from langchain.embeddings import HuggingFaceEmbeddings
embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/bert-base-nli-mean-tokens")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.77k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Save Embedding to ChromaDB

In [161]:
from langchain.vectorstores import Chroma

persist_directory = '/content/drive/MyDrive/code/Generate/chroma-vectorstore'

vectorstore = Chroma.from_documents(
    cleaned_docs,
    embedding=embed_model,
    persist_directory=persist_directory
)

## Initialize Text Generation Model

In [162]:
from langchain_huggingface import HuggingFacePipeline

llm = HuggingFacePipeline.from_model_id(
    model_id='google/flan-t5-large',
    task='text2text-generation',
    pipeline_kwargs={
        'max_new_tokens': 200,
        'max_length': 1024,
        'temperature': 0.7,
        'top_p': 0.9, # Consider top 90% of the probability distribution for next token
        'do_sample': True
    }
)

Device set to use cpu


## Create Template for Prompt

In [163]:
from langchain_core.prompts import ChatPromptTemplate

message = """
Should I {question} to my glasses? Consider the following guidelines:
Guidelines:
{guideline}

Answer:
"""

prompt_template = ChatPromptTemplate.from_messages([("human", message)])

## Initialize Retriver from vectorstore
Enabling searching for most similar documents

In [164]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 2}
)

## Chaining Prompt with LLM

In [165]:
from langchain_core.runnables import RunnablePassthrough

rag_chain = ({"guideline": retriever, "question": RunnablePassthrough()}
						| prompt_template
						| llm)

In [171]:
query = "Use strong detergent to clean glasses"

retrieved_docs = vectorstore.similarity_search(query, k=2)
print("Retrieved Documents:")
for doc in retrieved_docs:
    print(doc.page_content)

Retrieved Documents:
Title:  The  Beginner’s  Guide  to  Keeping  Your  Glasses  Clean  (and  Actually  Seeing  Clearly)   1.  Introduction   Glasses  are  like  win
drop  of  mild,  lotion-free  dish  soap  works  wonders.    3.  Gently  Rub  –  Use  your  fingers  to  clean  the  lenses  and  frame.    4.  Rinse  &  Dry  –  Pat


In [170]:
response = rag_chain.invoke(query)
print("LLM Response:", response)

Both `max_new_tokens` (=200) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


LLM Response: No
